# Perturbation Double Sweep
TJ Kim

1.6.22 

#### Summary:
- Sweep both Linf norm [0,1,0.5,1.0,2.0,4.0] for defense and attack
- Generate tables for adv_acc, adv_target, adv_miss

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


#### Import Relevant Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

#### Function - Calculate Mean without Diagonal

In [3]:
# Perform np.mean without the diagonal
def avg_nondiag(array2d):
    d1 = array2d.shape[0]
    d2 = array2d.shape[1]
    
    counter = 0
    val = 0
    
    for i1 in range(d1):
        for i2 in range(d2):
            if i1 != i2:
                counter+=1
                val += array2d[i1,i2]
    
    return val/counter

#### Generate Dummy Aggregator and Client

In [4]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "femnist"
args_.method = "FedEM_adv"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/22_01_09_fedavg_n80_benign/'
args_.validation = False
args_.num_user = 80

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=80)

==> Clients initialization..
===> Building data iterators..


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 719/719 [00:01<00:00, 452.41it/s]


===> Initializing clients..


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 719/719 [00:40<00:00, 17.54it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 4.133 | Train Acc: 0.592% |Test Loss: 4.130 | Test Acc: 0.614% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


#### Load Dataset From Client Validation

In [6]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.tensor(data_y)

# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

#### Import Experiment Name Information

Used later to loop through.

In [50]:
exp_names = ['weights/final/femnist/fig1_take2/FedEM_Benign/', 'weights/femnist/ep_sweep/fedem_defend_ep01/',
             'weights/femnist/ep_sweep/fedem_defend_ep02/', 'weights/femnist/ep_sweep/fedem_defend_ep04/']

# Benign Model Run
# exp_names = ['22_01_09_fedavg_n80_benign/']

# base = 'weights/cifar/'
train_item = 'train_client_weights.npy'

# Attack Perturbation Amount
atk_eps = [0, 0.1, 0.5, 1.0, 2.0, 4.0, 5, 6, 7, 8.0]

#### Set Measurement Metrics

In [51]:
num_victims = 10
num_exp = len(exp_names)
# Set Up Dictionaries -- list holds the adversary idx
exp_logs = {}

for j in range(num_exp):
    logs_adv = []

    for i in range(num_victims):
        adv_dict = {}
        adv_dict['orig_acc_transfers'] = None
        adv_dict['orig_similarities'] = None
        adv_dict['adv_acc_transfers'] = None
        adv_dict['adv_similarities_target'] = None
        adv_dict['adv_similarities_untarget'] = None
        adv_dict['adv_target'] = None
        adv_dict['adv_miss'] = None
        adv_dict['metric_alignment'] = None
        adv_dict['ib_distance_legit'] = None
        adv_dict['ib_distance_adv'] = None
        
        logs_adv += [adv_dict]
    
    exp_logs[j] = logs_adv


In [52]:
# Generate extra set of exp_logs one for each perturbation
exp_logs_list = []

for i in range(len(atk_eps)):
    exp_logs_list += [copy.deepcopy(exp_logs)]

#### Load Each Model and Perform Metric

In [59]:
# Inter Boundary Distance Metric
num_trials = 50
custom_batch_size = 500
weights_save = []

for j in range(num_exp):
    print('processing file', exp_names[j], '...')
    
    
    
    # Change name if need be
    args_.save_path = exp_names[j]

    # Import weights for aggregator
    aggregator.load_state(args_.save_path)

    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]
        
    weight_name = args_.save_path + train_item
    weights = np.load(weight_name)
    np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

    # Set model weights
    model_weights = []
    num_models = num_victims

    for i in range(num_models):
        model_weights += [weights[i]]

    weights_save += [model_weights]
        
    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0, w1, w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]
    
    for ep_idx in range(len(atk_eps)):
        # Run Measurements for both targetted and untargeted analysis
        victim_idxs = range(num_victims)

        

#         t2 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
#         t2.victim_idx = victim_idxs

        for adv_idx in victim_idxs:
            print("\t Adv idx:", adv_idx)
            # Perform Attacks
            
            dataloader2 = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train
            batch_size = min(custom_batch_size, dataloader2.y_data.shape[0])
            t1 = Transferer(models_list=models_test, dataloader=dataloader2)
            t1.generate_victims(victim_idxs)
            
            t1.atk_params = PGD_Params()
            t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                           target = 5, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                           step_size = 0.05, step_norm = "inf", eps = atk_eps[ep_idx], eps_norm = 2)

            t1.generate_advNN(adv_idx)
            t1.generate_xadv(atk_type = "pgd")
            t1.send_to_victims(victim_idxs)
            # t1.check_empirical_metrics(orig_flag = True)

            # Log Performance
            exp_logs_list[ep_idx][j][adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
            exp_logs_list[ep_idx][j][adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
            exp_logs_list[ep_idx][j][adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
            exp_logs_list[ep_idx][j][adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
            exp_logs_list[ep_idx][j][adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)
            # exp_logs_list[ep_idx][j][adv_idx]['metric_alignment'] = copy.deepcopy(t1.metric_alignment)

            # Miss attack
            t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                           target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                           step_size = 0.05, step_norm = "inf", eps = atk_eps[ep_idx], eps_norm = 2)
            t1.generate_xadv(atk_type = "pgd")
            t1.send_to_victims(victim_idxs)
            exp_logs_list[ep_idx][j][adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
            exp_logs_list[ep_idx][j][adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)
        
        del t1

    del models_test

processing file weights/final/femnist/fig1_take2/FedEM_Benign/ ...
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 0
	 Adv idx:

#### Obtain Metric Values and Make Table

In [60]:
# Make a pandas table across double sweep from list
def make_metric_table(exp_list, metric, row_names, col_names, avg_diag_flag = True):
    
    num_col1 = len(exp_list)
    num_col2 = len(exp_list[0])
    num_victims = len(exp_list[0][0])
    victim_idxs = range(num_victims)
    exp_values = {}
    
    final_table = np.zeros([num_col1, num_col2])
    
    for j in range(num_col1): # Attack perturbation amount
        for k in range(num_col2): # Defense perturbation amount (Experiment)
            orig_vals = np.zeros([num_victims, num_victims])
            
            for adv_idx in range(num_victims):
                for victim in range(num_victims):
                    curr_list = exp_list[j][k]
                    orig_vals[adv_idx,victim] = curr_list[victim_idxs[adv_idx]][metric][victim_idxs[victim]].data.tolist()
            
            if avg_diag_flag:
                final_table[j,k] = avg_nondiag(orig_vals)
            else:
                final_table[j,k] = np.mean(orig_vals)
    
    df = pd.DataFrame(final_table, columns = col_names, index = row_names)
    
    return df

In [61]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

In [62]:
row_names = ['A ep0','A ep0.1', 'A ep0.5', 'A ep1.0', 'A ep2.0',
             'A ep4.0', 'A ep5.0','A ep6.0', 'A ep7.0','A ep8.0']
col_names = ['D ep0.0', 'D ep0.1', 'D ep0.2', 'D ep0.3']
# col_names = ['G0','G0.5','G1']
# col_names = ['FedAvg G0']

name = 'adv_miss'
print(name)
make_metric_table(exp_logs_list, name, row_names, col_names, avg_diag_flag = True)

adv_miss


,D ep0.0,D ep0.1,D ep0.2,D ep0.4
A ep0,0.741829,0.592702,0.566032,0.527607
A ep0.1,0.741138,0.591656,0.564510,0.526623
A ep0.5,0.730265,0.586125,0.560445,0.522586
A ep1.0,0.719100,0.575057,0.552735,0.515905
A ep2.0,0.654334,0.547997,0.539185,0.504152
A ep4.0,0.456632,0.453357,0.476737,0.457537
A ep5.0,0.311810,0.378458,0.437895,0.431992
A ep6.0,0.175895,0.273307,0.377063,0.390509
A ep7.0,0.088780,0.177693,0.307042,0.348789
A ep8.0,0.041780,0.123900,0.247617,0.322412


In [64]:
name = 'adv_target'
print(name)
make_metric_table(exp_logs_list, name, row_names, col_names, avg_diag_flag = True)

adv_target


,D ep0.0,D ep0.1,D ep0.2,D ep0.4
A ep0,0.011417,0.010499,0.011610,0.011610
A ep0.1,0.011417,0.010499,0.011610,0.012651
A ep0.5,0.011417,0.010499,0.013346,0.013901
A ep1.0,0.011417,0.012319,0.013346,0.015720
A ep2.0,0.018687,0.018167,0.018218,0.018730
A ep4.0,0.056963,0.035992,0.029426,0.032739
A ep5.0,0.122291,0.070711,0.046777,0.045783
A ep6.0,0.242952,0.124182,0.074117,0.060791
A ep7.0,0.370364,0.181460,0.110276,0.093835
A ep8.0,0.385858,0.188842,0.117743,0.102551


In [27]:
weights_save[0] # Regular Training

[array([0.00, 0.00, 1.00]),
 array([0.00, 0.00, 1.00]),
 array([0.00, 0.00, 1.00]),
 array([0.32, 0.00, 0.68]),
 array([0.00, 0.00, 1.00]),
 array([0.15, 0.00, 0.85]),
 array([0.03, 0.00, 0.97]),
 array([0.10, 0.01, 0.89]),
 array([0.04, 0.00, 0.96]),
 array([0.00, 0.00, 1.00])]

In [28]:
weights_save[1]

[array([0.05, 0.00, 0.95]),
 array([0.07, 0.00, 0.93]),
 array([0.05, 0.00, 0.95]),
 array([0.57, 0.00, 0.43]),
 array([0.03, 0.00, 0.97]),
 array([0.46, 0.00, 0.54]),
 array([0.27, 0.00, 0.73]),
 array([0.44, 0.00, 0.56]),
 array([0.11, 0.00, 0.89]),
 array([0.01, 0.00, 0.99])]

In [29]:
weights_save[2] # Adv Training

[array([0.09, 0.00, 0.91]),
 array([0.12, 0.00, 0.88]),
 array([0.09, 0.00, 0.91]),
 array([0.65, 0.00, 0.35]),
 array([0.07, 0.00, 0.93]),
 array([0.50, 0.00, 0.50]),
 array([0.30, 0.00, 0.70]),
 array([0.53, 0.01, 0.46]),
 array([0.16, 0.00, 0.84]),
 array([0.07, 0.00, 0.93])]

In [30]:
weights_save[3]

[array([0.09, 0.00, 0.91]),
 array([0.13, 0.00, 0.87]),
 array([0.10, 0.00, 0.90]),
 array([0.64, 0.00, 0.36]),
 array([0.08, 0.00, 0.92]),
 array([0.48, 0.00, 0.52]),
 array([0.30, 0.00, 0.70]),
 array([0.51, 0.01, 0.47]),
 array([0.16, 0.00, 0.84]),
 array([0.08, 0.00, 0.92])]